# Exploratory data analysis

## 1. Dataset description

I will visualize the [DOHMH New York City Restaurant Inspection Results](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/rs6k-p7g6) dataset.

- The dataset is provided by the [New York City OpenData](https://opendata.cityofnewyork.us/).

- The dataset includes information about NYC restaurant names, locations, claims, inspections, violations, grades and adjudication. There are 26 variables, 19 of which are potentially useful and described here: 

    + CAMIS: an unique identifier of 10-digit integer for a restaurant. 
        
    + DBA: "doing business as", the name of the restaurant.

    + BORO: the Borough of the restaurant location.
    
    + BUILDING: the building of the restaurant location.
    
    + STREET: the street of the restaurant location.
    
    + ZIPCODE: the zipcode of the restaurant location.

    + PHONE: the phone number of the restaurant.

    + CUISINE DESCRIPTION: the cuisine type of the restaurant.

    + INSPECTION DATE: the date of the inspection. the date 1/1/1900 indicates the restaurant has not yet received any inspections.

    + ACTION: the action related with the inspection.

    + VIOLATION CODE: the violation code related with the inspection.

    + VIOLATION DESCRIPTION: the violation description related with the inspection.

    + CRITICAL FLAG: an indicator of critical violation.

    + INSPECTION TYPE: the type of inspection performed.

    + Latitude: the latitude of the restaurant location.

    + Longitude: the longitude of the restaurant location.

    + SCORE: score for the inspection, updated based on adjudication results.

    + GRADE: grade of the inspection. The grade is based on a two-step inspection process. A restaurant that doesn't receive a grade "A" on the initial inspection can be reinspected:
        
        - scoring less than 14 on the initial/re-inspection: grade "A";
        
        - scoring 14-27 on re-inspection: grade "B";
        
        - scoring 28 or more on re-inspection: grade "C";
        
        - reopening inspection or scoring more than 13 on the initial inspection: grade "P".

- The current dataset was collected between April, 2016 and March, 2020.

- The dataset is free to the public and was collected and provided by the Department of Health and Mental Hygiene (DOHMH). The purpose of this dataset is to collect recent restaurant inspection results.

- The dataset may contain missing values and errors.


## 2. Load the dataset

In [1]:
import pandas as pd
import numpy as np
import re
import json
import plotly.express as px


In [2]:
# dataset is downloaded from https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/rs6k-p7g6
data = pd.read_csv('../data/raw_data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

## 3. Explore the dataset

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402013 entries, 0 to 402012
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  402013 non-null  int64  
 1   DBA                    401647 non-null  object 
 2   BORO                   402013 non-null  object 
 3   BUILDING               401749 non-null  object 
 4   STREET                 402011 non-null  object 
 5   ZIPCODE                396529 non-null  float64
 6   PHONE                  401996 non-null  object 
 7   CUISINE DESCRIPTION    402013 non-null  object 
 8   INSPECTION DATE        402013 non-null  object 
 9   ACTION                 400732 non-null  object 
 10  VIOLATION CODE         396459 non-null  object 
 11  VIOLATION DESCRIPTION  393064 non-null  object 
 12  CRITICAL FLAG          393064 non-null  object 
 13  SCORE                  385257 non-null  float64
 14  GRADE                  203724 non-nu

In [4]:
# select columns of interest
isp_data = data[['CAMIS', 'DBA', 'BORO', 'BUILDING',
                 'STREET', 'ZIPCODE', 'PHONE', 
                 'CUISINE DESCRIPTION', 'INSPECTION TYPE',
                 'VIOLATION CODE', 'VIOLATION DESCRIPTION', 
                 'Latitude', 'Longitude', 'SCORE', 'GRADE']]

isp_data['INSPECTION DATE'] = pd.to_datetime(data['INSPECTION DATE'])

C:\Users\flizh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
isp_data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION TYPE,VIOLATION CODE,VIOLATION DESCRIPTION,Latitude,Longitude,SCORE,GRADE,INSPECTION DATE
0,50082518,PAPA JOHN'S,Brooklyn,1408,NEPTUNE AVE,11224.0,7182657272,Pizza,Calorie Posting / Re-inspection,16C,"Caloric content not posted on menus, menu boar...",40.579177,-73.982177,NaN,NaN,2018-12-13
1,50018573,TURCO,Manhattan,604,9TH AVE,10036.0,2125108666,Middle Eastern,Cycle Inspection / Initial Inspection,08A,Facility not vermin proof. Harborage or condit...,40.759235,-73.992026,48.0,NaN,2019-10-25
2,41600457,TABATA NOODLE RESTAURANT,Manhattan,540,9 AVENUE,10018.0,2122907681,Japanese,Cycle Inspection / Re-inspection,04L,Evidence of mice or live mice present in facil...,40.756993,-73.993654,28.0,C,2018-01-18
3,41405368,DYLAN MURPHY'S,Manhattan,1453,3 AVENUE,10028.0,2129889434,American,Cycle Inspection / Re-inspection,08A,Facility not vermin proof. Harborage or condit...,40.776321,-73.955788,11.0,A,2018-04-10
4,41594669,TAR PIT,Brooklyn,135,WOODPOINT ROAD,11211.0,6464699494,Café/Coffee/Tea,Cycle Inspection / Initial Inspection,04L,Evidence of mice or live mice present in facil...,40.717500,-73.941398,18.0,NaN,2019-02-14


In [6]:
isp_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402013 entries, 0 to 402012
Data columns (total 16 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   CAMIS                  402013 non-null  int64         
 1   DBA                    401647 non-null  object        
 2   BORO                   402013 non-null  object        
 3   BUILDING               401749 non-null  object        
 4   STREET                 402011 non-null  object        
 5   ZIPCODE                396529 non-null  float64       
 6   PHONE                  401996 non-null  object        
 7   CUISINE DESCRIPTION    402013 non-null  object        
 8   INSPECTION TYPE        400732 non-null  object        
 9   VIOLATION CODE         396459 non-null  object        
 10  VIOLATION DESCRIPTION  393064 non-null  object        
 11  Latitude               401591 non-null  float64       
 12  Longitude              401591 non-null  floa

In [7]:
print(f"There are {len(np.unique(isp_data['CAMIS']))} restaurants in this dataset.\n")

max_isp = isp_data.groupby(['CAMIS'])[['CUISINE DESCRIPTION']].count().sort_values(['CUISINE DESCRIPTION']).iloc[-1]

print(f"The restaurant with the maximal number of inspections is {isp_data[isp_data['CAMIS'] == max_isp.name].iloc[0]['DBA']} \
with {max_isp['CUISINE DESCRIPTION']} inspections.\n")

print(f"The Boroughs of the restaurant locations are {np.unique(isp_data['BORO'])}.\n\
There are {len(np.unique(isp_data[isp_data['BORO'] == '0']['CAMIS']))} restaurants without Borough information.\n")

print(f"There are {len(np.unique(isp_data['CUISINE DESCRIPTION']))} cuisine types in this dataset.\n\
The cuisine types are \n{np.unique(isp_data['CUISINE DESCRIPTION'])}.\n")

print(f"There are {sum(isp_data['INSPECTION DATE'].dt.year == 1900)} restaurants without any inspections.\n")

print(f"There are {len(set(isp_data['INSPECTION TYPE'])) - 1} inspection types:\n\
{list(set(isp_data['INSPECTION TYPE']))[1:]}.\n")

print(f"The inspection dates are between {np.min(isp_data[isp_data['INSPECTION DATE'].dt.year > 1900]['INSPECTION DATE'])} \
to {np.max(isp_data[isp_data['INSPECTION DATE'].dt.year > 1900]['INSPECTION DATE'])}.\n")

print(f"There are {len(set(isp_data['VIOLATION CODE'])) - 1} violation types in this dataset.\n\
The violation codes are \n{list(set(isp_data['VIOLATION CODE']))[1:]}.\n")

print(f"There are {len(set(isp_data['VIOLATION DESCRIPTION'])) - 1} violation description in this dataset.\n")

print(f"The range of score is from {np.min(isp_data['SCORE'])} to {np.max(isp_data['SCORE'])}. \
There are {sum(isp_data['SCORE'] < 0)} negative scores.\n")

print(f"There are {len(set(isp_data['GRADE'])) - 1} grade levels, including {list(set(isp_data['GRADE']))[1:]}.\n")

There are 27311 restaurants in this dataset.

The restaurant with the maximal number of inspections is ORCHID DYNASTY RESTAURANT with 98 inspections.

The Boroughs of the restaurant locations are ['0' 'Bronx' 'Brooklyn' 'Manhattan' 'Queens' 'Staten Island'].
There are 8 restaurants without Borough information.

There are 84 cuisine types in this dataset.
The cuisine types are 
['Afghan' 'African' 'American' 'Armenian' 'Asian' 'Australian'
 'Bagels/Pretzels' 'Bakery' 'Bangladeshi' 'Barbecue' 'Basque'
 'Bottled beverages, including water, sodas, juices, etc.' 'Brazilian'
 'Café/Coffee/Tea' 'Cajun' 'Californian' 'Caribbean' 'Chicken' 'Chilean'
 'Chinese' 'Chinese/Cuban' 'Chinese/Japanese' 'Continental' 'Creole'
 'Creole/Cajun' 'Czech' 'Delicatessen' 'Donuts' 'Eastern European'
 'Egyptian' 'English' 'Ethiopian' 'Filipino' 'French' 'Fruits/Vegetables'
 'German' 'Greek' 'Hamburgers' 'Hawaiian' 'Hotdogs' 'Hotdogs/Pretzels'
 'Ice Cream, Gelato, Yogurt, Ices' 'Indian' 'Indonesian' 'Iranian' 'Ir

## 4. Initial thoughts

1) What is the distribution of restaurant locations?
 
2) What is the distribution of the number of inspections per restaurant (per cuisine type)?
 
3) What is the distribution of cuisine types?
 
4) What is the distribution of inspection dates (per restaurant)?
 
5) What is the distribution of gaps between the initial inspection and re-inspection?
 
6) What is the distribution of grades?

7) What is the probability of getting a grade "A" on re-inspection?

8) What's the rank of Boroughs in the number of restaurants with grade "A"?

9) What's the rank of cuisine types in the number of restaurants with grade "A"?

10) I need to decide how to deal with restaurants that miss Borough or lon/lat information.

11) There are seven grade levels. I may need to regrade the scores into just three grade levels of "A", "B", "C".

## 5. Wrangling

In [8]:
mis_boro = np.unique(isp_data[isp_data['BORO'] == '0']['CAMIS'])
mis_lat = np.unique(isp_data[isp_data['Latitude'].isnull()]['CAMIS'])
mis_lon = np.unique(isp_data[isp_data['Longitude'].isnull()]['CAMIS'])

pd.DataFrame({'missing information': ['BORO', 'Latitude', 'Longitude'],
              'number of missing': [len(mis_boro), len(mis_lat), len(mis_lon)],
              'overlap with BORO': ['-', len(np.intersect1d(mis_boro, mis_lat)), len(np.intersect1d(mis_boro, mis_lon))],
              'overlap with Latitude': ['-', '-', len(np.intersect1d(mis_lat, mis_lon))]})


,missing information,number of missing,overlap with BORO,overlap with Latitude
0,BORO,8,-,-
1,Latitude,63,8,-
2,Longitude,63,8,63


In [9]:
# Check if it's possible to fill those missing location information based on other rows
dfg = isp_data.groupby(['CAMIS'])

ids_1 = []
for id in mis_boro:
    if any(dfg.get_group(id)['BORO'] != '0'):
        ids_1.append(id)
        
ids_2 = []
for id in mis_lat:
    if dfg.get_group(id)['Latitude'].notnull().any():
        ids_2.append(id)


In [10]:
# it's not possible to fill any of the rows with missing Borough information based related rows
len(ids_1)

0

In [11]:
# it's not possible to fill any of the rows with missing Lat/Lon information based related rows
len(ids_2)

0

> We still have the cuisine type information for those restaurants without location information, so I'll keep those rows.

In [12]:
mis_dba = np.unique(isp_data[isp_data['DBA'].isnull()]['CAMIS'])
mis_build = np.unique(isp_data[isp_data['BUILDING'].isnull()]['CAMIS'])
mis_zip = np.unique(isp_data[isp_data['ZIPCODE'].isnull()]['CAMIS'])

# Check if it's possible to fill those missing information based on other rows

ids_1 = []
for id in mis_dba:
    if dfg.get_group(id)['DBA'].notnull().any():
        ids_1.append(id)
        
ids_2 = []
for id in mis_build:
    if dfg.get_group(id)['BUILDING'].notnull().any():
        ids_2.append(id)
        
ids_3 = []
for id in mis_zip:
    if dfg.get_group(id)['ZIPCODE'].notnull().any():
        ids_3.append(id)


In [13]:
# it's not possible to fill any of the rows with missing DBA information based related rows
len(ids_1)

0

In [14]:
# it's not possible to fill any of the rows with missing building information based related rows
len(ids_2)

0

In [15]:
# it's not possible to fill any of the rows with missing zipcode information based related rows
len(ids_3)

0

In [16]:
# check whether information for each restaurant is coherent.
for id in dfg.groups.keys():
    df = dfg.get_group(id)
    for col in ['DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE', 
                 'CUISINE DESCRIPTION', 'Latitude', 'Longitude']:
        if not df[col].isnull().any() and len(np.unique(df[col])) > 1:
            print(id, col, np.unique(df[col]))

> Great, all those information is coherent.

In [17]:
# code inspection types into four groups: 0 : initial inspection, 1 : re-inspection, 2: reopening, -2: nan
code = np.full([isp_data.shape[0]], -2)
re_ips = isp_data['INSPECTION TYPE'].str.contains('Re-inspection|Second', flags=re.IGNORECASE, regex=True)
re_op = isp_data['INSPECTION TYPE'].str.contains('Reopening', flags=re.IGNORECASE, regex=True)
code[re_ips == True] = 1
code[re_ips == False] = 0
code[re_op == True] = 2
isp_data['INSPECTION CODE'] = code

# replace all nans with -2
isp_data = isp_data.fillna(-2)

C:\Users\flizh\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
# check whether scores and grades match. If not, re-assign the grades.
dfg = isp_data.groupby(['CAMIS'])
for id in dfg.groups.keys():
    df = dfg.get_group(id)
    for i in range(df.shape[0]):
        
        if df.iloc[i]['SCORE'] >= 0:
            
            if df.iloc[i]['INSPECTION CODE'] == 2:
                if df.iloc[i]['GRADE'] != 'P':
                    isp_data.loc[df.iloc[i].name, 'GRADE'] = 'P'
                    
            elif df.iloc[i]['SCORE'] < 14:
                if df.iloc[i]['GRADE'] != 'A':
                    isp_data.loc[df.iloc[i].name, 'GRADE'] = 'A'
                    
            elif df.iloc[i]['INSPECTION CODE'] == 1:
                if 14 <= df.iloc[i]['SCORE'] < 28:
                    if df.iloc[i]['GRADE'] != 'B':
                        isp_data.loc[df.iloc[i].name, 'GRADE'] = 'B'
                        
                elif df.iloc[i]['GRADE'] != 'C':
                    isp_data.loc[df.iloc[i].name, 'GRADE'] = 'C'
                    
            elif df.iloc[i]['INSPECTION CODE'] == 0:
                if df.iloc[i]['GRADE'] != 'P':
                    isp_data.loc[df.iloc[i].name, 'GRADE'] = 'P'
                
        elif df.iloc[i]['GRADE'] != -2 :
            isp_data.loc[df.iloc[i].name, 'GRADE'] = -2

In [19]:
# replace cuisine description with code
code_to_cuisine = dict(zip(range(len(set(isp_data['CUISINE DESCRIPTION']))), set(isp_data['CUISINE DESCRIPTION'])))
cuisine_to_code = dict(zip(set(isp_data['CUISINE DESCRIPTION']), range(len(set(isp_data['CUISINE DESCRIPTION'])))))
isp_data['CUISINE DESCRIPTION'].replace(cuisine_to_code, inplace=True)

# replace violation description with code
code_to_violation = dict(zip(range(len(set(isp_data['VIOLATION DESCRIPTION'])) - 1), list(set(isp_data['VIOLATION DESCRIPTION']))[: -1]))
violation_to_code = dict(zip(list(set(isp_data['VIOLATION DESCRIPTION']))[: -1], range(len(set(isp_data['VIOLATION DESCRIPTION'])) - 1)))
isp_data['VIOLATION DESCRIPTION'].replace(violation_to_code, inplace=True)

In [20]:
isp_data.head()

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION TYPE,VIOLATION CODE,VIOLATION DESCRIPTION,Latitude,Longitude,SCORE,GRADE,INSPECTION DATE,INSPECTION CODE
0,50082518,PAPA JOHN'S,Brooklyn,1408,NEPTUNE AVE,11224.0,7182657272,44,Calorie Posting / Re-inspection,16C,70,40.579177,-73.982177,-2.0,-2,2018-12-13,1
1,50018573,TURCO,Manhattan,604,9TH AVE,10036.0,2125108666,63,Cycle Inspection / Initial Inspection,08A,68,40.759235,-73.992026,48.0,P,2019-10-25,0
2,41600457,TABATA NOODLE RESTAURANT,Manhattan,540,9 AVENUE,10018.0,2122907681,78,Cycle Inspection / Re-inspection,04L,78,40.756993,-73.993654,28.0,C,2018-01-18,1
3,41405368,DYLAN MURPHY'S,Manhattan,1453,3 AVENUE,10028.0,2129889434,30,Cycle Inspection / Re-inspection,08A,68,40.776321,-73.955788,11.0,A,2018-04-10,1
4,41594669,TAR PIT,Brooklyn,135,WOODPOINT ROAD,11211.0,6464699494,52,Cycle Inspection / Initial Inspection,04L,78,40.717500,-73.941398,18.0,P,2019-02-14,0


In [21]:
# save restaurant information
rst_info = isp_data[['CAMIS', 'DBA', 'BORO', 'BUILDING',
                     'STREET', 'ZIPCODE', 'PHONE',
                     'CUISINE DESCRIPTION',
                     'Latitude', 'Longitude']].drop_duplicates()
rst_info.to_csv('../data/clean_data/nyc_restaurants_info.csv')

In [30]:
# save inspection information
isp_info = isp_data[['CAMIS', 'INSPECTION CODE', 'VIOLATION DESCRIPTION', 
                     'INSPECTION DATE', 'SCORE', 'GRADE']]\
[isp_data['SCORE'] >= 0].sort_values(['CAMIS', 'INSPECTION DATE'])

isp_info.to_csv('../data/clean_data/nyc_restaurants_grades.csv', index=False)

In [23]:
with open('../data/clean_data/code_to_cuisine.txt', 'w') as json_file:
    json.dump(code_to_cuisine, json_file)
    
with open('../data/clean_data/cuisine_to_code.txt', 'w') as json_file:
    json.dump(cuisine_to_code, json_file)

with open('../data/clean_data/code_to_violation.txt', 'w') as json_file:
    json.dump(code_to_violation, json_file)

with open('../data/clean_data/violation_to_code.txt', 'w') as json_file:
    json.dump(violation_to_code, json_file)

## 6. Research questions

1) Which Borough has the largest number of grade 'A' restaurants given a cuisine type?

2) What are the cuisine types with the top five largest number of grade 'A' in a given Borough?

3) What's the inspection history given a restaurant?

## 7. Data Analysis & Visualizations

In [38]:
df_1 = isp_data.groupby(['CAMIS'])[['INSPECTION DATE']].count()
df_1.columns = ['# of inspections per restaurant']
df_1.head()

,# of inspections per restaurant
CAMIS,
30075445,15
30112340,23
30191841,9
40356018,9
40356483,8


In [40]:
fig = px.histogram(df_1, x='# of inspections per restaurant')
fig.update_layout(title_text='The histogram of the number of inspections per restaurant')
fig.write_html('../img/inspection_per_restaurant.html', auto_open=True)


In [42]:
df_2 = data[['CUISINE DESCRIPTION', 'CAMIS']].drop_duplicates().groupby(['CUISINE DESCRIPTION'])[['CAMIS']].\
             count().sort_values('CAMIS', ascending=False).reset_index().iloc[:30]
df_2.columns = ['cuisine type', 'count']
df_2.head()

,cuisine type,count
0,American,5881
1,Chinese,2403
2,Café/Coffee/Tea,1879
3,Other,1447
4,Pizza,1207


In [43]:
fig = px.bar(df_2, x="cuisine type", y="count", barmode='group')

fig.update_layout(title_text='The number of restaurants per cuisine type (top 30)',
                  margin={"r":10,"t":50,"l":10,"b":200})

fig.write_html('../img/restaurants_per_cuisine_type.html', auto_open=True)

In [45]:
df_3 = pd.DataFrame(isp_info.groupby(['CAMIS']).apply(lambda x: x.iloc[-1]['GRADE']),
                    columns=['GRADE'])\
.reset_index().groupby(['GRADE'])[['CAMIS']].count().reset_index()
df_3.columns = ['grade', '# of restaurants']
df_3

,grade,# of restaurants
0,A,21695
1,B,1246
2,C,330
3,P,2759


In [46]:
fig = px.bar(df_3, x="grade", y="# of restaurants", barmode='group')
fig.update_layout(title_text='The number of restaurants per grade')

fig.write_html('../img/restaurant_per_grade.html', auto_open=True)

In [27]:
token = 'pk.eyJ1IjoiZmxpemhvdSIsImEiOiJjazg4Y2hjaW4wMjFlM3NtemhhNG90Z2ZzIn0.gxDbD64mpZbxE2HMjxEZng'

In [47]:
fig = px.scatter_mapbox(rst_info, lat="Latitude", lon="Longitude", hover_name="DBA", hover_data=["DBA"],
                        color_discrete_sequence=["fuchsia"], zoom=3, height=300)
fig.update_layout(mapbox_accesstoken=token)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.write_html('../img/restaurant_location.html', auto_open=True)

## 8. Summary and conclusions
